1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log (как вариант - использовать np.clip или np.where).

In [18]:
import numpy as np

In [19]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],  # квалификация репетитора
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1]) # поступил или нет ученик на специальность Математика

In [20]:
def calc_logloss(y, y_pred):
    np.clip(y_pred, 0, 1)
    err = np.mean(- y * np.log(y_pred) - (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [21]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [22]:
def eval_LR_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations + 1):
        z = np.dot(X, w) # log(p/(1-p))
        y_pred = sigmoid(z) # p [0, 1]
        err = calc_logloss(y, y_pred)
        w -= alpha * (1/n * np.dot(X.T, (y_pred - y)))
        if i % (iterations / 10) == 0:
            print(i, w, err)
    return w

In [23]:
def standard_scaler(x):
    res = (x - x.mean()) / x.std()
    return res

X_st = X.copy()
X_st[:, 2] = standard_scaler(X[:, 2])
X_st[:, 2]

array([-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
       -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735])

In [24]:
w = eval_LR_model(X_st, y, 1000, 1e-2)

100 [ 0.18484965 -0.70481989  0.79562879  1.18628976] 0.5521972941601363
200 [ 0.03068715 -0.71011925  0.97545893  1.1411597 ] 0.49256515417676405
300 [-0.08621623 -0.70278363  1.12058686  1.13528912] 0.45742460783827044
400 [-0.18040236 -0.70602545  1.23767464  1.14817577] 0.43449619084636665
500 [-0.25828078 -0.71688498  1.33435361  1.17418925] 0.4181995598483974
600 [-0.32439836 -0.7325847   1.41612844  1.20935475] 0.4056093071546024
700 [-0.38201827 -0.75130774  1.48682839  1.25071457] 0.39520237250559903
800 [-0.43345982 -0.77185692  1.54912899  1.29612888] 0.3861738764634576
900 [-0.48037073 -0.7934306   1.60492189  1.34407224] 0.3780860930070067
1000 [-0.52392631 -0.81548705  1.65556349  1.39346541] 0.37069115708188327


In [25]:
y_pred = sigmoid(np.dot(X_st, w))
print(y_pred.round(2))
print(y)
calc_logloss(y, y_pred)

[0.17 0.29 0.47 0.23 0.81 0.57 0.8  0.27 0.15 0.67]
[0 0 1 0 1 0 1 0 0 1]


0.37062015056058434

2. Подберите аргументы функции eval_LR_model для логистической регрессии таким образом, чтобы log loss был минимальным. Покажите влияние гиперпараметров на ошибку алгоритма (оптимально здесь использовать критерий остановки обучения).

In [30]:
w = eval_LR_model(X_st, y, 5000, 1e-2)
y_pred = sigmoid(np.dot(X_st, w))
print(y_pred.round(2))
print(y)
calc_logloss(y, y_pred)

500 [-0.25828078 -0.71688498  1.33435361  1.17418925] 0.4181995598483974
1000 [-0.52392631 -0.81548705  1.65556349  1.39346541] 0.37069115708188327
1500 [-0.71396854 -0.92362957  1.85844058  1.64233986] 0.3403953484499735
2000 [-0.88346761 -1.01988835  2.01108488  1.8749682 ] 0.3172688352762759
2500 [-1.04436589 -1.10312377  2.13318452  2.08651301] 0.29874725816355385
3000 [-1.19976932 -1.17526722  2.23324939  2.2792959 ] 0.283423473575798
3500 [-1.35072503 -1.23834983  2.3162308   2.45635264] 0.27041142790035855
4000 [-1.49772867 -1.29403688  2.38551189  2.62028546] 0.2591262667735922
4500 [-1.64109655 -1.34363239  2.44358927  2.77316742] 0.2491683632514042
5000 [-1.78106882 -1.38815503  2.49238023  2.91663433] 0.2402564002441388
[0.06 0.16 0.53 0.1  0.91 0.56 0.94 0.16 0.05 0.8 ]
[0 0 1 0 1 0 1 0 0 1]


0.24023947827288014

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, на выходе - массив y_pred_proba).

In [ ]:
def calc_pred_proba()

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, на выходе - массив y_pred).


In [ ]:
def calc_pred()